In [1]:
import numpy as np
import cv2 as cv
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader,Dataset
import os
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import torchvision.models as models

In [2]:
class mydataset(Dataset):
    def __init__(self, path,transform=None):
        filenames=[]
        labels1 = []
        labels2 = []
        with open(path, 'r') as file:
            for line in file:
                parts = line.strip().split()  # 移除行尾的换行符并按空格分割
                filename = parts[0]
                label1 = int(parts[1])
                label2 = int(parts[2])
                filenames.append(filename)
                labels1.append(label1)
                labels2.append(label2)
       
        self.all_image_paths = filenames
        self.all_image_labels = labels1
        self.transform=transform

    def __getitem__(self, index):
        img = Image.open('images_jpegs_255/'+self.all_image_paths[index]).convert('RGB')
        img=self.transform(img)
        label=self.all_image_labels[index]
        label=torch.tensor(label,dtype=torch.float32)
        return img, label

    def __len__(self):
        return len(self.all_image_paths)

In [3]:
data_transforms = transforms.Compose([
    transforms.Resize(256),    # 将图片短边缩放至256，长宽比保持不变：
    transforms.CenterCrop(224),   #将图片从中心切剪成3*224*224大小的图片
    transforms.ToTensor()          #把图片进行归一化，并把数据转换成Tensor类型
])

In [4]:
train_data=mydataset('train.txt',data_transforms)
test_data=mydataset('test.txt',data_transforms)
train_dataloader=DataLoader(train_data,batch_size=64,shuffle=True)
test_dataloader=DataLoader(test_data,batch_size=64,shuffle=True)

train_data_size=len(train_data)
test_data_size=len(test_data)

In [5]:
resnet50 = models.resnet50(pretrained=True)
num_ftrs = resnet50.fc.in_features 
for param in resnet50.parameters():
    param.requires_grad = False #False：冻结模型的参数，也就是采用该模型已经训练好的原始参数。只需要训练我们自己定义的Linear层
#保持in_features不变，修改out_features=2
resnet50.fc = nn.Sequential(nn.Linear(num_ftrs,2),
                           
                
                            nn.LogSoftmax(dim=1))

c:\Users\armstrong\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\armstrong\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
model=resnet50
model.cuda()
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [7]:
#损失函数和优化器
loss_fn=nn.CrossEntropyLoss()
loss_fn=loss_fn.cuda()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

In [8]:
total_train_step=0
total_test_step=0
for i in range(30):
    print('第{}轮训练'.format(i+1))
    model.train()
    for data in train_dataloader:
        imgs,targets=data
        targets=targets.long()
        imgs=imgs.cuda()
        targets=targets.cuda()
        outputs=model(imgs)
        loss=loss_fn(outputs,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_step=total_train_step+1
    
    model.eval()
    total_test_loss=0
    total_accuracy=0
    with torch.no_grad():
        for data in test_dataloader:
            imgs,targets=data
            imgs=imgs.cuda()
            targets=targets.long()
            targets=targets.cuda()
            outputs=model(imgs)
            loss=loss_fn(outputs,targets)
            total_test_loss=total_test_loss+loss
            accuracy=(outputs.argmax(1)==targets).sum()
            total_accuracy=total_accuracy+accuracy
    print("整体测试集上的Loss:{}".format(total_test_loss))
    print('测试集上的正确率:{}'.format(total_accuracy/test_data_size))
    total_test_step=total_test_step+1

torch.save(model,'resnet50_1.pth')

第1轮训练
整体测试集上的Loss:2.3843326568603516
测试集上的正确率:0.38181817531585693
第2轮训练
整体测试集上的Loss:1.410889744758606
测试集上的正确率:0.4999999701976776
第3轮训练
整体测试集上的Loss:1.417443871498108
测试集上的正确率:0.48181816935539246
第4轮训练
整体测试集上的Loss:1.2581803798675537
测试集上的正确率:0.6363636255264282
第5轮训练
整体测试集上的Loss:1.66848623752594
测试集上的正确率:0.6181818246841431
第6轮训练
整体测试集上的Loss:1.2457573413848877
测试集上的正确率:0.6090908646583557
第7轮训练
整体测试集上的Loss:1.466440200805664
测试集上的正确率:0.5454545021057129
第8轮训练
整体测试集上的Loss:1.2377653121948242
测试集上的正确率:0.6545454263687134
第9轮训练
整体测试集上的Loss:1.0905135869979858
测试集上的正确率:0.7636363506317139
第10轮训练
整体测试集上的Loss:1.0393022298812866
测试集上的正确率:0.7727272510528564
第11轮训练
整体测试集上的Loss:1.2651060819625854
测试集上的正确率:0.6545454263687134
第12轮训练
整体测试集上的Loss:1.0367134809494019
测试集上的正确率:0.7272726893424988
第13轮训练
整体测试集上的Loss:1.1022939682006836
测试集上的正确率:0.6909090876579285
第14轮训练
整体测试集上的Loss:1.0563160181045532
测试集上的正确率:0.7363635897636414
第15轮训练
整体测试集上的Loss:1.0042892694473267
测试集上的正确率:0.7999999523162842
第16轮训练
整体测试集上的Loss:1.0